### Mask Classification Refactoring
22년 여름 방학에 딥러닝을 갓 배우고, 만들었던 마스크 분류기를 리팩토링

하게 된 이유 : 그 동안 많은 경험을 위해 닥치는 대로 딥러닝을 사용해보고, 여러 지식을 습득하기 바밨었으나, 이제는 이전 작품을 다시 리팩토링 및 코드를 최대한 gpt의 도움 없이 짜보면서 내실을 다지고 싶었기 때문.

In [ ]:
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from PIL import Image
import glob
import sys, os
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from collections import OrderedDict
import pickle
import torch
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
from activation import relu
from commons import col2im, im2col
from layers import batch_normalization, convolution, fc_layer, pooling
from losses import cross_entropy_loss, softmax_with_loss, softmax
from models import vgg6
from optimizer import adam
from Trainer import trainer
from utils import get_grad, graph, shuffle_dataset, split_dataset, visualize_result, earlystop

In [ ]:
print(f'GPU 사용 가능 여부: {torch.cuda.is_available()}')
device = "cuda" if torch.cuda.is_available() else "CPU"

In [ ]:
# 난수 생성을 위한 시드 설정
SEED = 777

# CPU 환경을 위한 PyTorch 난수 생성기에 시드 설정
torch.manual_seed(SEED)

# 현재 GPU를 위한 PyTorch 난수 생성기에 시드 설정
torch.cuda.manual_seed(SEED)

# cuDNN의 결정적(deterministic) 알고리즘 사용 여부 설정
# False로 설정할 경우, 비결정적 알고리즘이 허용되어 성능이 향상될 수 있지만, 재현성이 감소할 수 있음
torch.backends.cudnn.deterministic = False

# cuDNN에서 최적의 알고리즘을 자동으로 찾도록 설정
# True로 설정할 경우, 고정된 입력 크기에 대해 더 빠른 성능을 제공하지만, 다양한 크기의 입력에서는 성능 저하가 발생할 수 있음
torch.backends.cudnn.benchmark = True

In [ ]:
# 데이터셋 디렉토리 설정 해야함

In [ ]:
class MaskDataset(Dataset):
    """
    마스크 착용 여부를 구별하는 이미지 데이터셋 클래스.

    이 클래스는 주어진 디렉토리에서 마스크 착용 여부를 구분하는 이미지 데이터셋을 로드하고,
    albumentations 라이브러리를 사용한 이미지 변환을 적용한다.

    Parameters:
        data_dir (str): 데이터셋이 위치한 디렉토리 경로.
        mode (str): 데이터셋 모드 ('train', 'val', 'test').
        transform (albumentations.Compose): 이미지에 적용할 변환.
    """
    def __init__(self, data_dir, mode, transform=None):
        self.all_data = sorted(glob.glob(os.path.join(data_dir, mode, '*', '*')))
        self.transform = transform
        
    
    def __getitem__(self, index):
        """
        인덱스에 해당하는 데이터를 반환합니다.

        Parameters:
            index (int): 데이터셋에서 가져올 샘플의 인덱스.

        Returns:
            tuple: (변환된 이미지, 레이블).
        """
        data_path = self.all_data[index]
        img = Image.open(data_path)
        label = 0 if os.path.basename(data_path).startswith("Mask") else 1
        
        if self.transform:
            img = self.transform(image=np.array(img))["image"]
            
        return img, label
    
    def __len__(self):
        """
        데이터셋의 전체 길이를 반환합니다.

        Returns:
            int: 데이터셋의 전체 길이.
        """
        return len(self.all_data)

In [ ]:
# albumentations을 사용한 데이터 변환 정의
data_transforms = {
    'train': A.Compose([
        A.RandomRotate90(),
        A.Flip(),
        A.Transpose(),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=.1),
            A.Blur(blur_limit=3, p=.1),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.Sharpen(),
            A.Emboss(),
            A.RandomBrightnessContrast(),
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
        ToTensorV2()
    ]),
    'val': A.Compose([
        A.Resize(224, 224),
        ToTensorV2()
    ])
}

In [ ]:
# 데이터셋 로딩, 실제 데이터셋 받아서 경로 수정 필요
data_dir = './data'
split_dir = './splitdata'

# 데이터셋 분할
split_dataset.split_dataset(data_dir, split_dir)

train_dataset = MaskDataset(split_dir, 'train', transform=data_transforms['train'])
val_dataset = MaskDataset(split_dir, 'val', transform=data_transforms['val'])
test_dataset = MaskDataset(split_dir, 'test', transform=data_transforms['val'])

In [ ]:
"""
torch의 DataLoader의 shuffle 기능을 사용하지 않고, 커스텀한 shuffle 함수로 사용하기 위한 코드
"""

x = []
t = []

for img, label in train_dataset:
    x.append(img.numpy())
    t.append(label)

x = np.array(x)
# x 배열이 4차원이 아니라면 차원을 증가시킵니다.
# 예를 들어, x 배열이 (샘플 수, 높이, 너비, 채널 수) 형태가 아닌 경우
if x.ndim < 4:
    x = np.expand_dims(x, axis=1)  # 채널 차원 추가

t = np.array(t)

# 데이터셋 섞기
x_shuffled, t_shuffled = shuffle_dataset.shuffle_dataset(x, t)

# TensorDataset으로 변환
tensor_x = torch.Tensor(x_shuffled)
tensor_t = torch.Tensor(t_shuffled)

shuffled_trainset = TensorDataset(tensor_x, tensor_t)

In [ ]:
batch_size = 32
num_epochs = 300
learning_rate = 0.0001
num_workers = 0

In [ ]:
# 데이터로더
train_loader = DataLoader(shuffled_trainset, batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True, num_workers=num_workers)

In [ ]:
model = vgg6.VGG6

In [ ]:
trainer = trainer.Trainer(model, x_train_loader = train_loader, x_test_loader = test_loader,
                          epochs=num_epochs, mini_batch_size=batch_size,
                          optimizer=adam.Adam, evaluate_sample_num_per_epoch=5)